#### Install CUML


#### Mount to Gdrive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Intrusion Detection System

In [2]:
# Import neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### About Dataset

Link : https://www.kaggle.com/datasets/cicdataset/cicids2017/data

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/Preprocess_CICIDS2017_Model1.csv')
df.head()

,Active Max,Idle Min,Active Mean,Idle Mean,Fwd IAT Std,PSH Flag Count,Idle Max,URG Flag Count,SYN Flag Count,Fwd PSH Flags,...,Fwd Packets/s,Bwd IAT Min,ECE Flag Count,RST Flag Count,Fwd IAT Min,Bwd PSH Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Label
0,-0.149326,-0.338993,-0.125734,-0.351926,-0.338929,-0.651648,-0.356868,-0.323661,-0.220701,-0.220701,...,2.435196,-0.116412,-0.015603,-0.015569,-0.118943,0.0,0.0,0.001237,0.001565,0
1,-0.149326,-0.338993,-0.125734,-0.351926,-0.338929,-0.651648,-0.356868,3.089652,-0.220701,-0.220701,...,-0.220941,-0.116412,-0.015603,-0.015569,-0.118943,0.0,0.0,0.001236,0.001579,0
2,-0.149326,-0.338993,-0.125734,-0.351926,-0.338929,-0.651648,-0.356868,3.089652,-0.220701,-0.220701,...,-0.180315,-0.116412,-0.015603,-0.015569,-0.118943,0.0,0.0,0.001236,0.001579,0
3,-0.149326,-0.338993,-0.125734,-0.351926,-0.338929,-0.651648,-0.356868,3.089652,-0.220701,-0.220701,...,-0.139186,-0.116412,-0.015603,-0.015569,-0.118943,0.0,0.0,0.001236,0.001579,0
4,-0.149326,-0.338993,-0.125734,-0.351926,-0.338929,-0.651648,-0.356868,-0.323661,-0.220701,-0.220701,...,2.435196,-0.116412,-0.015603,-0.015569,-0.118943,0.0,0.0,0.001237,0.001565,0


In [4]:
Total = df[' Label'].count()
Attack = sum(df[' Label']==1)

print(f'Out of {Total} records, {Attack} are attacks recorded (Percentage: {round(Attack/Total*100,2)}%)')

Out of 2830743 records, 557646 are attacks recorded (Percentage: 19.7%)


In [5]:
# Split Feature and Target Variable

x = df.drop([' Label'], axis=1)
y = df[' Label']


#### PCA

In [6]:
from sklearn.decomposition import PCA
import pandas as pd

# Assuming x is your input DataFrame

# Apply PCA
pca = PCA(n_components=10)
x_reduced = pca.fit_transform(x)

# Convert the reduced data (x_reduced) into a DataFrame
reduced_df = pd.DataFrame(x_reduced, columns=[f'Component_{i+1}' for i in range(x_reduced.shape[1])])

# Get the indices of the top features
top_feature_indices = pca.components_.argsort(axis=1)[:, ::-1][:, :10]

# Get the names of the top features
selected_feature_names = []
for indices_row in top_feature_indices:
    selected_feature_names.append(x.columns[indices_row])

# Map the column names of reduced_df to original feature names
reduced_df.columns = [' - '.join(feature_names) for feature_names in selected_feature_names]

# Create a DataFrame to store original features in each component
original_features_df = pd.DataFrame(selected_feature_names, index=[f'Component_{i+1}' for i in range(x_reduced.shape[1])])

# Print the reduced DataFrame with column names mapped to original feature names
print("Reduced DataFrame:")
reduced_df.head()



Reduced DataFrame:


,Idle Max - Flow IAT Max - Fwd IAT Max - Idle Mean - Idle Min - Flow IAT Std - Fwd IAT Std - Flow Duration - Bwd IAT Min - Fwd IAT Min,Active Max - Active Mean - Active Min - Active Std - PSH Flag Count - Init_Win_bytes_forward - Fwd Packet Length Std - Flow Duration - Init_Win_bytes_backward - RST Flag Count,SYN Flag Count - Fwd PSH Flags - ACK Flag Count - Fwd Packets/s - Active Mean - Active Max - Active Min - URG Flag Count - Active Std - Bwd IAT Min,ECE Flag Count - RST Flag Count - Fwd PSH Flags - SYN Flag Count - Init_Win_bytes_forward - Fwd Packet Length Std - PSH Flag Count - Fwd Packets/s - ACK Flag Count - Fwd IAT Std,Bwd IAT Min - Fwd IAT Min - Flow IAT Std - PSH Flag Count - SYN Flag Count - Fwd PSH Flags - Init_Win_bytes_forward - Active Std - Active Max - Active Mean,URG Flag Count - ACK Flag Count - Active Min - Active Mean - Active Max - Fwd IAT Min - Init_Win_bytes_backward - Bwd IAT Min - RST Flag Count - ECE Flag Count,FIN Flag Count - Fwd Packets/s - Active Mean - Active Min - Active Max - Active Std - ECE Flag Count - RST Flag Count - Fwd PSH Flags - SYN Flag Count,Active Std - Active Max - Flow Duration - URG Flag Count - FIN Flag Count - Init_Win_bytes_backward - ACK Flag Count - Bwd IAT Min - Bwd PSH Flags - Bwd URG Flags,Active Min - Init_Win_bytes_forward - Fwd Packet Length Std - SYN Flag Count - Fwd PSH Flags - PSH Flag Count - Init_Win_bytes_backward - Flow IAT Max - Fwd IAT Max - Fwd IAT Std,Fwd Packets/s - Init_Win_bytes_backward - FIN Flag Count - Init_Win_bytes_forward - PSH Flag Count - Fwd IAT Min - Bwd IAT Min - Active Min - Active Mean - Flow IAT Std
0,-1.081845,-0.835748,1.076285,-0.006216,-0.328079,0.681858,0.580518,0.062780,-0.023521,1.231468
1,-1.086912,-0.875544,0.982630,-0.079237,-0.579050,2.187241,-1.628391,0.240373,-0.004272,-0.478097
2,-1.088417,-0.879221,0.992184,-0.078380,-0.580607,2.184178,-1.621231,0.240288,-0.004544,-0.453223
3,-1.090112,-0.882597,1.001834,-0.077468,-0.582513,2.181763,-1.618235,0.240517,-0.004800,-0.423668
4,-1.081844,-0.835761,1.076295,-0.006219,-0.328082,0.681886,0.580542,0.062787,-0.023521,1.231456


In [ ]:
# To save this pca Object for future use
import pickle
# Save the PCA object to a file
with open('pca_object_model1.pkl', 'wb') as f:
    pickle.dump(pca, f)


reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 10 columns):
 #   Column                                                                                                                                                                                      Dtype  
---  ------                                                                                                                                                                                      -----  
 0    Idle Max -  Flow IAT Max -  Fwd IAT Max - Idle Mean -  Idle Min -  Flow IAT Std -  Fwd IAT Std -  Flow Duration -  Bwd IAT Min -  Fwd IAT Min                                              float64
 1    Active Max - Active Mean -  Active Min -  Active Std -  PSH Flag Count - Init_Win_bytes_forward -  Fwd Packet Length Std -  Flow Duration -  Init_Win_bytes_backward -  RST Flag Count     float64
 2    SYN Flag Count - Fwd PSH Flags -  ACK Flag Count - Fwd Packets/s - Active Mea

# Training and Testing


In [7]:
from sklearn.model_selection import train_test_split

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.25, random_state=42)



## Supervised Learning


In [ ]:
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Evaluate Models
kernal_evals = dict()

def evaluate_classification(model, name, X_train, X_test, y_train, y_test):
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    train_precision = precision_score(y_train, model.predict(X_train))
    test_precision = precision_score(y_test, model.predict(X_test))

    train_recall = recall_score(y_train, model.predict(X_train))
    test_recall = recall_score(y_test, model.predict(X_test))

    kernal_evals[str(name)] = [train_accuracy, test_accuracy, train_precision, test_precision, train_recall, test_recall]
    print("Training Accuracy " + str(name) + " {}  Test Accuracy ".format(train_accuracy*100) + str(name) + " {}".format(test_accuracy*100))
    print("Training Precision " + str(name) + " {}  Test Precision ".format(train_precision*100) + str(name) + " {}".format(test_precision*100))
    print("Training Recall " + str(name) + " {}  Test Recall ".format(train_recall*100) + str(name) + " {}".format(test_recall*100))

    confusion_matrix_result = confusion_matrix(y_test, model.predict(X_test))
    print("Confusion Matrix:")
    print(confusion_matrix_result)

    with open(name + '.pkl', 'wb') as f:
        pickle.dump(model, f)
        print(f"Model : {name} Saved !!! ")

# Convert data to scikit-learn format (optional)
# If your data is already in numpy arrays or pandas DataFrame/Series, you can skip this step

# x_train_sklearn = x_train_cudf.to_pandas()
# x_test_sklearn = x_test_cudf.to_pandas()
# y_train_sklearn = y_train_cudf.to_array()
# y_test_sklearn = y_test_cudf.to_array()

# Create models
sklearn_random_forest = RandomForestClassifier()
sklearn_svc = SVC()
sklearn_gaussian_nb = GaussianNB()
sklearn_k_neighbors = KNeighborsClassifier()
sklearn_logistic_regression = LogisticRegression()

# Train models
sklearn_random_forest.fit(x_train, y_train)
evaluate_classification(sklearn_random_forest, "RandomForest", x_train, x_test, y_train, y_test)

sklearn_svc.fit(x_train, y_train)
evaluate_classification(sklearn_svc, "SVC", x_train, x_test, y_train, y_test)

# sklearn_gaussian_nb.fit(x_train, y_train)
# evaluate_classification(sklearn_gaussian_nb, "GaussianNB", x_train, x_test, y_train, y_test)

sklearn_k_neighbors.fit(x_train, y_train)
evaluate_classification(sklearn_k_neighbors, "KNeighbors", x_train, x_test, y_train, y_test)

sklearn_logistic_regression.fit(x_train, y_train)
evaluate_classification(sklearn_logistic_regression, "LogisticRegression", x_train, x_test, y_train, y_test)


Training Accuracy RandomForest 99.95709017704188  Test Accuracy RandomForest 99.83749855161751
Training Precision RandomForest 99.79226609042394  Test Precision RandomForest 99.4982990766416
Training Recall RandomForest 99.99019082435355  Test Recall RandomForest 99.67924393212573
Confusion Matrix:
[[567314    702]
 [   448 139222]]
Model : RandomForest Saved !!! 


In [8]:
import pickle
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Evaluate Models
kernal_evals = dict()

def evaluate_classification(model, name, X_train, X_test, y_train, y_test):
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))

    train_precision = precision_score(y_train, model.predict(X_train))
    test_precision = precision_score(y_test, model.predict(X_test))

    train_recall = recall_score(y_train, model.predict(X_train))
    test_recall = recall_score(y_test, model.predict(X_test))

    kernal_evals[str(name)] = [train_accuracy, test_accuracy, train_precision, test_precision, train_recall, test_recall]
    print("Training Accuracy " + str(name) + " {}  Test Accuracy ".format(train_accuracy*100) + str(name) + " {}".format(test_accuracy*100))
    print("Training Precision " + str(name) + " {}  Test Precision ".format(train_precision*100) + str(name) + " {}".format(test_precision*100))
    print("Training Recall " + str(name) + " {}  Test Recall ".format(train_recall*100) + str(name) + " {}".format(test_recall*100))

    confusion_matrix_result = confusion_matrix(y_test, model.predict(X_test))
    print("Confusion Matrix:")
    print(confusion_matrix_result)

    with open(name + '.pkl', 'wb') as f:
        pickle.dump(model, f)
        print(f"Model : {name} Saved !!! ")

# Convert data to scikit-learn format (optional)
# If your data is already in numpy arrays or pandas DataFrame/Series, you can skip this step

# x_train_sklearn = x_train_cudf.to_pandas()
# x_test_sklearn = x_test_cudf.to_pandas()
# y_train_sklearn = y_train_cudf.to_array()
# y_test_sklearn = y_test_cudf.to_array()

# Create models
sklearn_k_neighbors = KNeighborsClassifier()
sklearn_logistic_regression = LogisticRegression()

sklearn_k_neighbors.fit(x_train, y_train)
evaluate_classification(sklearn_k_neighbors, "KNeighbors", x_train, x_test, y_train, y_test)

sklearn_logistic_regression.fit(x_train, y_train)
evaluate_classification(sklearn_logistic_regression, "LogisticRegression", x_train, x_test, y_train, y_test)

Training Accuracy KNeighbors 99.81667943912952  Test Accuracy KNeighbors 99.7496064638837
Training Precision KNeighbors 99.34082589796385  Test Precision KNeighbors 99.11597093603078
Training Recall KNeighbors 99.7306065420024  Test Recall KNeighbors 99.6198181427651
Confusion Matrix:
[[566775   1241]
 [   531 139139]]
Model : KNeighbors Saved !!! 
Training Accuracy LogisticRegression 83.93580577440926  Test Accuracy LogisticRegression 83.91009572041838
Training Precision LogisticRegression 74.53622189900227  Test Precision LogisticRegression 74.59491402645926
Training Recall LogisticRegression 27.953758110513522  Test Recall LogisticRegression 28.016753776759508
Confusion Matrix:
[[554689  13327]
 [100539  39131]]
Model : LogisticRegression Saved !!! 
